In [1]:
import pandas as pd
import sqlalchemy as sql
from sql_functions import get_sql_config
from sql_functions import get_engine
from sql_functions import push_to_database
from update import update_sql_table
import psycopg2

csv-File importieren.

In [2]:
nrw_2022 = pd.read_csv("data/nrw_2022.csv")
nrw_2022.head(20)
nrw_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8832 entries, 0 to 8831
Data columns (total 10 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Einzelplan                          8832 non-null   int64 
 1   Kapitel                             8832 non-null   int64 
 2   Gruppe                              8832 non-null   int64 
 3   Zählnummer                          8832 non-null   int64 
 4   Funktion                            8832 non-null   int64 
 5   Ansatz 2022 in Euro                 8832 non-null   object
 6   Ansatz 2021 in Euro                 8832 non-null   object
 7   Ist 2020 in Euro                    8832 non-null   object
 8   Verpflichtungsermächtigung in Euro  8832 non-null   object
 9   Zweckbestimmung                     8832 non-null   object
dtypes: int64(5), object(5)
memory usage: 690.1+ KB


Columns names anpassen, Sonderzeichen entfernen, in tausend Euro entfernen

In [3]:
nrw_2022.columns = nrw_2022.columns.str.replace(" ", "_")
nrw_2022.columns = nrw_2022.columns.str.lower()
nrw_2022.rename(columns={"ansatz_2022_t€":"ansatz_2022_euro"}, inplace=True)
nrw_2022.tail()

,einzelplan,kapitel,gruppe,zählnummer,funktion,ansatz_2022_in_euro,ansatz_2021_in_euro,ist_2020_in_euro,verpflichtungsermächtigung_in_euro,zweckbestimmung
8827,11,280,981,11,891,59.800,72.700,"51.076,27",0,Erstattung von Beihilfen in Krankheitsfällen f...
8828,11,280,981,40,891,0,0,"0,00",0,Erstattung von Versorgungsbezügen und Nachvers...
8829,12,70,981,0,891,1.027.000,675.000,"675.000,00",0,Verechnung zwischen Kapiteln
8830,12,641,981,0,891,0,0,"0,00",0,Ausgaben für Tätigkeiten der Bezirksregierungen
8831,3,310,989,0,891,0,0,"4.814.721,37",0,Haushaltstechnische Verrechnungen


Umbenennung der Namen im Columns

In [4]:
nrw_2022.rename(columns = {'einzelplan':'ep', 'zählnummer':'counter'}, inplace = True)
nrw_2022.head()

,ep,kapitel,gruppe,counter,funktion,ansatz_2022_in_euro,ansatz_2021_in_euro,ist_2020_in_euro,verpflichtungsermächtigung_in_euro,zweckbestimmung
0,20,10,11,0,821,19.145.000.000,19.150.000.000,"18.573.403.592,52",0,Lohnsteuer (Landesanteil)
1,20,10,12,0,821,6.065.000.000,5.013.000.000,"5.158.883.256,87",0,Veranlagte Einkommensteuer (Landesanteil)
2,20,10,13,0,821,3.466.000.000,2.256.000.000,"2.687.978.508,35",0,Nicht veranlagte Steuern vom Ertrag (ohne Abge...
3,20,10,14,0,821,4.081.000.000,2.885.000.000,"2.413.309.357,33",0,Körperschaftsteuer (Landesanteil)
4,20,10,15,10,821,21.534.600.000,18.604.300.000,"18.574.904.810,13",0,Umsatzsteuer (Landesanteil)


droppen von columns

In [5]:
nrw_2022.drop('ist_2020_in_euro', inplace=True, axis=1)

drop von column

In [6]:
nrw_2022.drop('ansatz_2021_in_euro', inplace=True, axis=1)
nrw_2022.head()


,ep,kapitel,gruppe,counter,funktion,ansatz_2022_in_euro,verpflichtungsermächtigung_in_euro,zweckbestimmung
0,20,10,11,0,821,19.145.000.000,0,Lohnsteuer (Landesanteil)
1,20,10,12,0,821,6.065.000.000,0,Veranlagte Einkommensteuer (Landesanteil)
2,20,10,13,0,821,3.466.000.000,0,Nicht veranlagte Steuern vom Ertrag (ohne Abge...
3,20,10,14,0,821,4.081.000.000,0,Körperschaftsteuer (Landesanteil)
4,20,10,15,10,821,21.534.600.000,0,Umsatzsteuer (Landesanteil)


drop column

In [7]:
nrw_2022.drop('verpflichtungsermächtigung_in_euro', inplace=True, axis=1)
nrw_2022.head()

,ep,kapitel,gruppe,counter,funktion,ansatz_2022_in_euro,zweckbestimmung
0,20,10,11,0,821,19.145.000.000,Lohnsteuer (Landesanteil)
1,20,10,12,0,821,6.065.000.000,Veranlagte Einkommensteuer (Landesanteil)
2,20,10,13,0,821,3.466.000.000,Nicht veranlagte Steuern vom Ertrag (ohne Abge...
3,20,10,14,0,821,4.081.000.000,Körperschaftsteuer (Landesanteil)
4,20,10,15,10,821,21.534.600.000,Umsatzsteuer (Landesanteil)


check-up

In [8]:
nrw_2022

,ep,kapitel,gruppe,counter,funktion,ansatz_2022_in_euro,zweckbestimmung
0,20,10,11,0,821,19.145.000.000,Lohnsteuer (Landesanteil)
1,20,10,12,0,821,6.065.000.000,Veranlagte Einkommensteuer (Landesanteil)
2,20,10,13,0,821,3.466.000.000,Nicht veranlagte Steuern vom Ertrag (ohne Abge...
3,20,10,14,0,821,4.081.000.000,Körperschaftsteuer (Landesanteil)
4,20,10,15,10,821,21.534.600.000,Umsatzsteuer (Landesanteil)
...,...,...,...,...,...,...,...
8827,11,280,981,11,891,59.800,Erstattung von Beihilfen in Krankheitsfällen f...
8828,11,280,981,40,891,0,Erstattung von Versorgungsbezügen und Nachvers...
8829,12,70,981,0,891,1.027.000,Verechnung zwischen Kapiteln
8830,12,641,981,0,891,0,Ausgaben für Tätigkeiten der Bezirksregierungen


column eingefügt mit Jahr/Wert=2022

In [9]:
nrw_2022.insert(loc=6,
          column='year',
          value=2022)

nrw_2022.head()

,ep,kapitel,gruppe,counter,funktion,ansatz_2022_in_euro,year,zweckbestimmung
0,20,10,11,0,821,19.145.000.000,2022,Lohnsteuer (Landesanteil)
1,20,10,12,0,821,6.065.000.000,2022,Veranlagte Einkommensteuer (Landesanteil)
2,20,10,13,0,821,3.466.000.000,2022,Nicht veranlagte Steuern vom Ertrag (ohne Abge...
3,20,10,14,0,821,4.081.000.000,2022,Körperschaftsteuer (Landesanteil)
4,20,10,15,10,821,21.534.600.000,2022,Umsatzsteuer (Landesanteil)


Column hinzugefügt mit state_id/wert= 10 (NRW)

In [10]:
nrw_2022.insert(loc=7,
          column='state_id',
          value=10)

nrw_2022.head()

,ep,kapitel,gruppe,counter,funktion,ansatz_2022_in_euro,year,state_id,zweckbestimmung
0,20,10,11,0,821,19.145.000.000,2022,10,Lohnsteuer (Landesanteil)
1,20,10,12,0,821,6.065.000.000,2022,10,Veranlagte Einkommensteuer (Landesanteil)
2,20,10,13,0,821,3.466.000.000,2022,10,Nicht veranlagte Steuern vom Ertrag (ohne Abge...
3,20,10,14,0,821,4.081.000.000,2022,10,Körperschaftsteuer (Landesanteil)
4,20,10,15,10,821,21.534.600.000,2022,10,Umsatzsteuer (Landesanteil)


check-up

In [11]:
nrw_2022

,ep,kapitel,gruppe,counter,funktion,ansatz_2022_in_euro,year,state_id,zweckbestimmung
0,20,10,11,0,821,19.145.000.000,2022,10,Lohnsteuer (Landesanteil)
1,20,10,12,0,821,6.065.000.000,2022,10,Veranlagte Einkommensteuer (Landesanteil)
2,20,10,13,0,821,3.466.000.000,2022,10,Nicht veranlagte Steuern vom Ertrag (ohne Abge...
3,20,10,14,0,821,4.081.000.000,2022,10,Körperschaftsteuer (Landesanteil)
4,20,10,15,10,821,21.534.600.000,2022,10,Umsatzsteuer (Landesanteil)
...,...,...,...,...,...,...,...,...,...
8827,11,280,981,11,891,59.800,2022,10,Erstattung von Beihilfen in Krankheitsfällen f...
8828,11,280,981,40,891,0,2022,10,Erstattung von Versorgungsbezügen und Nachvers...
8829,12,70,981,0,891,1.027.000,2022,10,Verechnung zwischen Kapiteln
8830,12,641,981,0,891,0,2022,10,Ausgaben für Tätigkeiten der Bezirksregierungen


Reihenfolge nach upload_budget angepasst -> wird nocht geändert

In [12]:
nrw_2022 = nrw_2022[['ep', 'kapitel', 'zweckbestimmung', 'ansatz_2022_in_euro', 'gruppe', 'counter', 'funktion', 'year', 'state_id']]
nrw_2022.head()

,ep,kapitel,zweckbestimmung,ansatz_2022_in_euro,gruppe,counter,funktion,year,state_id
0,20,10,Lohnsteuer (Landesanteil),19.145.000.000,11,0,821,2022,10
1,20,10,Veranlagte Einkommensteuer (Landesanteil),6.065.000.000,12,0,821,2022,10
2,20,10,Nicht veranlagte Steuern vom Ertrag (ohne Abge...,3.466.000.000,13,0,821,2022,10
3,20,10,Körperschaftsteuer (Landesanteil),4.081.000.000,14,0,821,2022,10
4,20,10,Umsatzsteuer (Landesanteil),21.534.600.000,15,10,821,2022,10


check-up

In [13]:
nrw_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8832 entries, 0 to 8831
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ep                   8832 non-null   int64 
 1   kapitel              8832 non-null   int64 
 2   zweckbestimmung      8832 non-null   object
 3   ansatz_2022_in_euro  8832 non-null   object
 4   gruppe               8832 non-null   int64 
 5   counter              8832 non-null   int64 
 6   funktion             8832 non-null   int64 
 7   year                 8832 non-null   int64 
 8   state_id             8832 non-null   int64 
dtypes: int64(7), object(2)
memory usage: 621.1+ KB


Umwandlung der Werte im Columns um weiteres vorgehen zu ermöglichen 

In [15]:
nrw_2022['ep'] = nrw_2022['ep'].astype(str)
nrw_2022['ep'] = nrw_2022['ep'].apply(lambda x:x.zfill(2))
nrw_2022['kapitel'] = nrw_2022['kapitel'].astype(str)
#sh_2022['titel'] = sh_2022['titel'].astype(str)
#sh_2022['titel'] = sh_2022['titel'].apply(lambda x:x.zfill(5))
nrw_2022['counter'] = nrw_2022['counter'].astype(str)
nrw_2022['counter'] = nrw_2022['counter'].apply(lambda x:x.zfill(2))
nrw_2022['funktion'] = nrw_2022['funktion'].astype(str)
nrw_2022['funktion'] = nrw_2022['funktion'].apply(lambda x:x.zfill(3))
nrw_2022

,ep,kapitel,zweckbestimmung,ansatz_2022_in_euro,gruppe,counter,funktion,year,state_id
0,20,10,Lohnsteuer (Landesanteil),19.145.000.000,11,00,821,2022,10
1,20,10,Veranlagte Einkommensteuer (Landesanteil),6.065.000.000,12,00,821,2022,10
2,20,10,Nicht veranlagte Steuern vom Ertrag (ohne Abge...,3.466.000.000,13,00,821,2022,10
3,20,10,Körperschaftsteuer (Landesanteil),4.081.000.000,14,00,821,2022,10
4,20,10,Umsatzsteuer (Landesanteil),21.534.600.000,15,10,821,2022,10
...,...,...,...,...,...,...,...,...,...
8827,11,280,Erstattung von Beihilfen in Krankheitsfällen f...,59.800,981,11,891,2022,10
8828,11,280,Erstattung von Versorgungsbezügen und Nachvers...,0,981,40,891,2022,10
8829,12,70,Verechnung zwischen Kapiteln,1.027.000,981,00,891,2022,10
8830,12,641,Ausgaben für Tätigkeiten der Bezirksregierungen,0,981,00,891,2022,10


Kapitel überarbeitet mit nullziffer-> vergelich roh daten 

In [16]:
nrw_2022['kapitel'] = nrw_2022['kapitel'].apply(lambda x:x.zfill(3))

In [17]:
nrw_2022

,ep,kapitel,zweckbestimmung,ansatz_2022_in_euro,gruppe,counter,funktion,year,state_id
0,20,010,Lohnsteuer (Landesanteil),19.145.000.000,11,00,821,2022,10
1,20,010,Veranlagte Einkommensteuer (Landesanteil),6.065.000.000,12,00,821,2022,10
2,20,010,Nicht veranlagte Steuern vom Ertrag (ohne Abge...,3.466.000.000,13,00,821,2022,10
3,20,010,Körperschaftsteuer (Landesanteil),4.081.000.000,14,00,821,2022,10
4,20,010,Umsatzsteuer (Landesanteil),21.534.600.000,15,10,821,2022,10
...,...,...,...,...,...,...,...,...,...
8827,11,280,Erstattung von Beihilfen in Krankheitsfällen f...,59.800,981,11,891,2022,10
8828,11,280,Erstattung von Versorgungsbezügen und Nachvers...,0,981,40,891,2022,10
8829,12,070,Verechnung zwischen Kapiteln,1.027.000,981,00,891,2022,10
8830,12,641,Ausgaben für Tätigkeiten der Bezirksregierungen,0,981,00,891,2022,10


Summen mit '.' werden ersetzt (tausender punkt)

In [18]:
nrw_2022['ansatz_2022_in_euro'] = nrw_2022['ansatz_2022_in_euro'].str.replace(".", "")

/var/folders/rp/8t70135j493dk68133cwmvx40000gn/T/ipykernel_4086/1298728458.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  nrw_2022['ansatz_2022_in_euro'] = nrw_2022['ansatz_2022_in_euro'].str.replace(".", "")


check-up

In [19]:
nrw_2022

,ep,kapitel,zweckbestimmung,ansatz_2022_in_euro,gruppe,counter,funktion,year,state_id
0,20,010,Lohnsteuer (Landesanteil),19145000000,11,00,821,2022,10
1,20,010,Veranlagte Einkommensteuer (Landesanteil),6065000000,12,00,821,2022,10
2,20,010,Nicht veranlagte Steuern vom Ertrag (ohne Abge...,3466000000,13,00,821,2022,10
3,20,010,Körperschaftsteuer (Landesanteil),4081000000,14,00,821,2022,10
4,20,010,Umsatzsteuer (Landesanteil),21534600000,15,10,821,2022,10
...,...,...,...,...,...,...,...,...,...
8827,11,280,Erstattung von Beihilfen in Krankheitsfällen f...,59800,981,11,891,2022,10
8828,11,280,Erstattung von Versorgungsbezügen und Nachvers...,0,981,40,891,2022,10
8829,12,070,Verechnung zwischen Kapiteln,1027000,981,00,891,2022,10
8830,12,641,Ausgaben für Tätigkeiten der Bezirksregierungen,0,981,00,891,2022,10


umwandeln in int

In [20]:
nrw_2022['ansatz_2022_in_euro'] = nrw_2022['ansatz_2022_in_euro'].astype(int)

checkp-up

In [21]:
nrw_2022

,ep,kapitel,zweckbestimmung,ansatz_2022_in_euro,gruppe,counter,funktion,year,state_id
0,20,010,Lohnsteuer (Landesanteil),19145000000,11,00,821,2022,10
1,20,010,Veranlagte Einkommensteuer (Landesanteil),6065000000,12,00,821,2022,10
2,20,010,Nicht veranlagte Steuern vom Ertrag (ohne Abge...,3466000000,13,00,821,2022,10
3,20,010,Körperschaftsteuer (Landesanteil),4081000000,14,00,821,2022,10
4,20,010,Umsatzsteuer (Landesanteil),21534600000,15,10,821,2022,10
...,...,...,...,...,...,...,...,...,...
8827,11,280,Erstattung von Beihilfen in Krankheitsfällen f...,59800,981,11,891,2022,10
8828,11,280,Erstattung von Versorgungsbezügen und Nachvers...,0,981,40,891,2022,10
8829,12,070,Verechnung zwischen Kapiteln,1027000,981,00,891,2022,10
8830,12,641,Ausgaben für Tätigkeiten der Bezirksregierungen,0,981,00,891,2022,10


alle Werte >=400 werden zu negative Zahlen umgewandelt

In [22]:
nrw_2022.loc[nrw_2022['gruppe'] >= 400, 'ansatz_2022_in_euro'] = nrw_2022['ansatz_2022_in_euro']* -1
nrw_2022['gruppe'] = nrw_2022['gruppe'].astype(str)
nrw_2022['gruppe'] = nrw_2022['gruppe'].apply(lambda x:x.zfill(3))

letzter check up

In [23]:
nrw_2022


,ep,kapitel,zweckbestimmung,ansatz_2022_in_euro,gruppe,counter,funktion,year,state_id
0,20,010,Lohnsteuer (Landesanteil),19145000000,011,00,821,2022,10
1,20,010,Veranlagte Einkommensteuer (Landesanteil),6065000000,012,00,821,2022,10
2,20,010,Nicht veranlagte Steuern vom Ertrag (ohne Abge...,3466000000,013,00,821,2022,10
3,20,010,Körperschaftsteuer (Landesanteil),4081000000,014,00,821,2022,10
4,20,010,Umsatzsteuer (Landesanteil),21534600000,015,10,821,2022,10
...,...,...,...,...,...,...,...,...,...
8827,11,280,Erstattung von Beihilfen in Krankheitsfällen f...,-59800,981,11,891,2022,10
8828,11,280,Erstattung von Versorgungsbezügen und Nachvers...,0,981,40,891,2022,10
8829,12,070,Verechnung zwischen Kapiteln,-1027000,981,00,891,2022,10
8830,12,641,Ausgaben für Tätigkeiten der Bezirksregierungen,0,981,00,891,2022,10


In [24]:
nrw_2022.rename(columns = {'funktion':'fkz', 'ansatz_2022_in_euro':'amount'}, inplace = True)

In [25]:
nrw_2022.head()

,ep,kapitel,zweckbestimmung,amount,gruppe,counter,fkz,year,state_id
0,20,010,Lohnsteuer (Landesanteil),19145000000,011,00,821,2022,10
1,20,010,Veranlagte Einkommensteuer (Landesanteil),6065000000,012,00,821,2022,10
2,20,010,Nicht veranlagte Steuern vom Ertrag (ohne Abge...,3466000000,013,00,821,2022,10
3,20,010,Körperschaftsteuer (Landesanteil),4081000000,014,00,821,2022,10
4,20,010,Umsatzsteuer (Landesanteil),21534600000,015,10,821,2022,10


In [26]:
get_sql_config()
engine = get_engine()

In [27]:
#push_to_database(nrw_2022, "budget_all", engine, "capstone_public_budgeting")

The budget_all table was imported successfully.


In [2]:
df_ep_kapitel = pd.read_csv('data/ep_kapitel_nrw_22.csv',encoding= 'UTF-8',dtype={'ep':str,'kapitel':str})

In [3]:
df_ep_kapitel['year'] = 2022
df_ep_kapitel['state_id'] = 10
df_ep_kapitel

,ep_txt,ep,kapitel,kapitel_txt,year,state_id
0,Landtag,01,010,Landtag,2022,10
1,Landtag,01,100,Landesbeauftragte für Datenschutz und Informat...,2022,10
2,Landtag,01,900,"Versorgung der Beamtinnen und Beamten, Richte...",2022,10
3,Ministerpräsident,02,010,Ministerpräsident,2022,10
4,Ministerpräsident,02,020,Allgemeine Bewilligungen,2022,10
...,...,...,...,...,...,...
244,Ministerium für Landwirtschaft und Verbraucher...,15,090,Zuschüsse der Europäischen Union (EU),2022,10
245,Ministerium für Landwirtschaft und Verbraucher...,15,100,Landwirtschaftskammer Nordrhein-Westfalen und ...,2022,10
246,Ministerium für Landwirtschaft und Verbraucher...,15,200,Landesforstverwaltung,2022,10
247,Ministerium für Landwirtschaft und Verbraucher...,15,400,Nordrhein-Westfälisches Landgestüt,2022,10


In [5]:
update_sql_table(df_ep_kapitel[['ep','kapitel','kapitel_txt','year','state_id']],'kapitel')
update_sql_table(df_ep_kapitel[['ep','ep_txt','year','state_id']],'einzelplaene')

The kapitel table was imported successfully.


Wird eigentlich nicht mehr benötigt

In [32]:
#push_to_database(df_ep_kapitel[['ep','kapitel','kapitel_txt','year','state_id']], "kapitel", engine, "capstone_public_budgeting")

The kapitel table was imported successfully.


In [33]:
#push_to_database(df_ep_kapitel[['ep','ep_txt','year','state_id']].drop_duplicates(),
#                "einzelplaene",
#                engine, 
#                "capstone_public_budgeting"
#                )

The einzelplaene table was imported successfully.
